# Bronx and Brooklyn Neighborhood Comparison

## Introduction

## Table of Content

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Webscrape and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in The Bronx and Brooklyn</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


## 1. Download and Explore Dataset.

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
# Load the data.
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [1]:
# Take a look at the data.
newyork_data

#### All of the relevant data is in the features key. Define a new variable that includes the features data.

In [5]:
neighborhood_data = newyork_data['features']

In [6]:
# Take a look at the first item in the list.
neighborhood_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

### Transform the data into a pandas dataframe.
transform the data from nested Python dictionaries into a pandas dataframe.

In [7]:
# Define the dataframe columns.
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


#### Loop through the data and fill the dataframe.

In [9]:
for data in neighborhood_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                    'Neighborhood': neighborhood_name,
                                    'Latitude': neighborhood_lat,
                                    'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
# Change the name of the dataframe.
bxbk_df = neighborhoods

In [12]:
bxbk_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
# I only want to look at neighborhoods in the Bronx and Brooklyn. 
# Drop rows with Manhattan, Queens, and Staten Island.
bxbk_df.drop(neighborhoods.index[bxbk_df['Borough'] == 'Manhattan'], inplace = True)
bxbk_df.drop(neighborhoods.index[bxbk_df['Borough'] == 'Queens'], inplace = True)
bxbk_df.drop(neighborhoods.index[bxbk_df['Borough'] == 'Staten Island'], inplace = True)

In [14]:
# Check to make sure  Bronx and Brooklyn are left in the dataframe.
bxbk_df.Borough.unique()

array(['Bronx', 'Brooklyn'], dtype=object)

In [15]:
bxbk_df.reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Bronx,Woodlawn,40.898273,-73.867315
7,Bronx,Norwood,40.877224,-73.879391
8,Bronx,Williamsbridge,40.881039,-73.857446
9,Bronx,Baychester,40.866858,-73.835798


In [16]:
# Check to see how many neighborhoods there are.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(bxbk_df['Borough'].unique()),
        bxbk_df.shape[0]))

The dataframe has 2 boroughs and 122 neighborhoods.


### Use the geopy library to get the latitude and longitude values of New York City.

In [17]:
city = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


### Create a Map of New York City with the neighborhoods plotted onto it.

In [18]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(bxbk_df['Latitude'], bxbk_df['Longitude'], bxbk_df['Borough'], bxbk_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Utilize the Foursquare API in order to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version.

In [19]:
CLIENT_ID = 'VW4IYSJWZOV5MEUTYKEWPEINY3TZWYE0GJHVKNUQUSJ5DVFW' # your Foursquare ID
CLIENT_SECRET = 'NORPTKWY0S0MX3LH53PPQPJ410DHEFFJMVU055BAKPE1QAH4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: VW4IYSJWZOV5MEUTYKEWPEINY3TZWYE0GJHVKNUQUSJ5DVFW
CLIENT_SECRET:NORPTKWY0S0MX3LH53PPQPJ410DHEFFJMVU055BAKPE1QAH4


### Exploring the Pelham Bay neighborhood in our dataframe.

In [20]:
bxbk_df.loc[37, 'Neighborhood']

'Pelham Bay'

In [21]:
# Get the latitude and longitude values for Pelham Bay.
hood_lat = bxbk_df.loc[37, 'Latitude']
hood_long = bxbk_df.loc[37, 'Longitude']

hood_name = bxbk_df.loc[37, 'Neighborhood']
radius=750
LIMIT=100

print('Latitude and Longitude values of {} are {}, {}'.format(hood_name,
                                                               hood_lat,
                                                               hood_long))

Latitude and Longitude values of Pelham Bay are 40.85064140940335, -73.8320737824047


### Get the top 100 venues that are located in Pelham Bay within the radius of 750 meters.
First, create the GET request URL.

In [22]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    hood_lat, 
    hood_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VW4IYSJWZOV5MEUTYKEWPEINY3TZWYE0GJHVKNUQUSJ5DVFW&client_secret=NORPTKWY0S0MX3LH53PPQPJ410DHEFFJMVU055BAKPE1QAH4&v=20180605&ll=40.85064140940335,-73.8320737824047&radius=750&limit=100'

In [23]:
# Send the GET request and examie the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c925a1d9fb6b73bdd355700'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4c249bffa852c9285f52e36c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bagels_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d179941735',
         'name': 'Bagel Shop',
         'pluralName': 'Bagel Shops',
         'primary': True,
         'shortName': 'Bagels'}],
       'id': '4c249bffa852c9285f52e36c',
       'location': {'address': '3101 Westchester Ave',
        'cc': 'US',
        'city': 'Bronx',
        'country': 'United States',
        'distance': 190,
        'formattedAddress': ['3101 Westchester Ave',
         'Bronx, NY 10461',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.8

### All the information needed is in the items key. First write a function that extracts the category of venues.

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json data and structure it into a pandas dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Empire Bagels,Bagel Shop,40.849392,-73.830527
1,Zeppieri & Sons Italian Bakery,Bakery,40.847119,-73.832057
2,"iLoveKickboxing - Bronx, NY",Gym,40.852871,-73.828085
3,George's Restaurant,Diner,40.846945,-73.832184
4,Pruzzo's,Deli / Bodega,40.846831,-73.832663


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

53 venues were returned by Foursquare.


In [35]:
# Plot out a map of the venues of Pelham Bay.
map_pb = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_pb)  
    
map_pb

## 2. Explore Neighborhoods in The Bronx and Brooklyn

In [36]:
# Function to repeat the same process to all the neighborhoods in Toronto.
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
# Run the above function on each neighborhood and create a new dataframe.
bxbk_venues = getNearbyVenues(names=bxbk_df['Neighborhood'],
                                 latitudes=bxbk_df['Latitude'],
                                 longitudes=bxbk_df['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Ger

In [38]:
bxbk_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,40.889283,-73.843310,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Rite Aid,40.889062,-73.842993,Pharmacy
4,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop


In [39]:
bxbk_venues.shape

(6569, 7)

### Checkout how many venues were returned for each neighborhood.

In [40]:
bxbk_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,44,44,44,44,44,44
Bath Beach,77,77,77,77,77,77
Bay Ridge,100,100,100,100,100,100
Baychester,66,66,66,66,66,66
Bedford Park,55,55,55,55,55,55
Bedford Stuyvesant,82,82,82,82,82,82
Belmont,96,96,96,96,96,96
Bensonhurst,67,67,67,67,67,67
Bergen Beach,7,7,7,7,7,7


In [41]:
print('There are {} uniques categories.'.format(len(bxbk_venues['Venue Category'].unique())))

There are 358 uniques categories.


# 3. Analyze Each Neighborhood.

In [42]:
# Get dummy variables.
bxbk_onehot = pd.get_dummies(bxbk_venues[['Venue Category']], prefix='', prefix_sep='')

# Add neighborhood column back to dataframe.
bxbk_onehot['Neighborhood'] = bxbk_venues['Neighborhood']

# Move neighborhood column to the first column.
fixed_columns = [bxbk_onehot.columns[-1]] + list(bxbk_onehot.columns[:-1])
bxbk_onehot = bxbk_onehot[fixed_columns]

bxbk_onehot.head()

,Zoo,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake 

In [43]:
bxbk_onehot.shape

(6569, 358)

### Group rows by neighborhood and by taking the mean of the frequency of occurance of each category.

In [45]:
bxbk_grouped = bxbk_onehot.groupby('Neighborhood').mean().reset_index()
bxbk_grouped.head()

,Neighborhood,Zoo,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Cafeteria,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Sake 

In [46]:
bxbk_grouped.shape

(122, 358)

### Print out each neighborhood with the top 5 most common venues.

In [47]:
num_top_venues = 5

for hood in bxbk_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bxbk_grouped[bxbk_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
               venue  freq
0         Donut Shop  0.09
1        Pizza Place  0.09
2     Sandwich Place  0.07
3  Martial Arts Dojo  0.05
4        Supermarket  0.05


----Bath Beach----
                  venue  freq
0           Pizza Place  0.06
1    Chinese Restaurant  0.06
2                  Bank  0.05
3                Bakery  0.05
4  Cantonese Restaurant  0.05


----Bay Ridge----
                venue  freq
0  Italian Restaurant  0.07
1                 Spa  0.07
2         Pizza Place  0.06
3      Cosmetics Shop  0.04
4    Greek Restaurant  0.04


----Baychester----
                    venue  freq
0             Bus Station  0.05
1  Furniture / Home Store  0.05
2                Pharmacy  0.05
3                    Bank  0.03
4           Shopping Mall  0.03


----Bedford Park----
                venue  freq
0         Pizza Place  0.07
1       Deli / Bodega  0.07
2         Supermarket  0.07
3               Diner  0.05
4  Mexican Restaurant  0.05


----Bedford Stuyvesant----

                  venue  freq
0  Caribbean Restaurant  0.10
1           Bus Station  0.08
2           Pizza Place  0.06
3           Coffee Shop  0.06
4         Deli / Bodega  0.06


----Flatlands----
                  venue  freq
0  Fast Food Restaurant  0.09
1              Pharmacy  0.09
2  Caribbean Restaurant  0.06
3          Liquor Store  0.06
4        Sandwich Place  0.06


----Fordham----
                 venue  freq
0   Spanish Restaurant  0.06
1          Pizza Place  0.06
2  Fried Chicken Joint  0.06
3    Mobile Phone Shop  0.05
4           Shoe Store  0.05


----Fort Greene----
                venue  freq
0         Coffee Shop  0.06
1  Italian Restaurant  0.05
2         Pizza Place  0.04
3                 Bar  0.04
4        Cocktail Bar  0.03


----Fort Hamilton----
                 venue  freq
0          Pizza Place  0.11
1   Italian Restaurant  0.05
2  Japanese Restaurant  0.05
3    Convenience Store  0.04
4         Cocktail Bar  0.04


----Fulton Ferry----
                 

           venue  freq
0    Bus Station  0.08
1     Donut Shop  0.08
2  Deli / Bodega  0.08
3       Pharmacy  0.06
4    Supermarket  0.06


----Pelham Parkway----
                venue  freq
0         Pizza Place  0.14
1      Sandwich Place  0.14
2  Italian Restaurant  0.10
3        Liquor Store  0.05
4  Frozen Yogurt Shop  0.05


----Port Morris----
                   venue  freq
0     Chinese Restaurant  0.11
1     Spanish Restaurant  0.07
2                    Gym  0.07
3  Performing Arts Venue  0.04
4               Pharmacy  0.04


----Prospect Heights----
                venue  freq
0                 Bar  0.07
1     Thai Restaurant  0.04
2        Cocktail Bar  0.04
3  Mexican Restaurant  0.04
4           Wine Shop  0.04


----Prospect Lefferts Gardens----
                  venue  freq
0  Caribbean Restaurant  0.13
1           Pizza Place  0.06
2                  Café  0.05
3                Bakery  0.05
4          Liquor Store  0.04


----Prospect Park South----
                  ve

### Put the most common venues into a dataframe.

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
# Create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bxbk_grouped['Neighborhood']

for ind in np.arange(bxbk_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bxbk_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Donut Shop,Sandwich Place,Martial Arts Dojo,Supermarket,Pharmacy,Bank,Fast Food Restaurant,Food,Bus Station
1,Bath Beach,Pizza Place,Chinese Restaurant,Cantonese Restaurant,Bakery,Bank,Donut Shop,Coffee Shop,Bubble Tea Shop,Pharmacy,Japanese Restaurant
2,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Cosmetics Shop,Bagel Shop,Bar,Greek Restaurant,Chinese Restaurant,Bakery,Donut Shop
3,Baychester,Pharmacy,Furniture / Home Store,Bus Station,Bank,Coffee Shop,Discount Store,Supermarket,Bakery,Department Store,Spanish Restaurant
4,Bedford Park,Deli / Bodega,Supermarket,Pizza Place,Sandwich Place,Mexican Restaurant,Diner,Chinese Restaurant,Food Truck,Donut Shop,Garden


# 4. Cluster Neighborhoods.
Run _k_-means to cluster the neighborhoods into 5 clusters.

In [50]:
# Set the number of clusters
kclusters = 5

bxbk_clustering = bxbk_grouped.drop('Neighborhood', 1)

# Run the k-means clustering.
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bxbk_clustering)

# Check the cluster labels generated for each row in the dataframe.
kmeans.labels_[0:10]

array([2, 2, 0, 2, 2, 0, 0, 2, 1, 0], dtype=int32)

### Create a new dataframe that includes the clusters as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bxbk_merged = bxbk_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bxbk_merged = bxbk_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bxbk_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,4,Supermarket,Caribbean Restaurant,Pharmacy,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Health & Beauty Service,Gas Station,Fried Chicken Joint,Donut Shop
1,Bronx,Co-op City,40.874294,-73.829939,2,Mattress Store,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Supermarket,Men's Store,Spanish Restaurant,Pharmacy,Mobile Phone Shop,Moving Target
2,Bronx,Eastchester,40.887556,-73.827806,4,Bus Station,Caribbean Restaurant,Fast Food Restaurant,Women's Store,Grocery Store,Pizza Place,Diner,Bus Stop,Shopping Mall,Business Service
3,Bronx,Fieldston,40.895437,-73.905643,3,Plaza,Playground,Park,Café,River,Home Service,Art Gallery,Bus Station,Coffee Shop,Pizza Place
4,Bronx,Riverdale,40.890834,-73.912585,0,Pizza Place,Japanese Restaurant,Mexican Restaurant,Bar,Bank,Bagel Shop,Diner,Plaza,Italian Restaurant,Café


### Finally visualize the resulting clusters on the map.

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bxbk_merged['Latitude'], bxbk_merged['Longitude'], bxbk_merged['Neighborhood'], bxbk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine the Clusters.

## Cluster 1

In [53]:
bxbk_merged.loc[bxbk_merged['Cluster Labels'] == 0, bxbk_merged.columns[[1] + list(range(5, bxbk_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,Pizza Place,Japanese Restaurant,Mexican Restaurant,Bar,Bank,Bagel Shop,Diner,Plaza,Italian Restaurant,Café
5,Kingsbridge,Bar,Pizza Place,Mexican Restaurant,Sandwich Place,Supermarket,Park,Diner,Donut Shop,Café,Trail
7,Woodlawn,Pizza Place,Deli / Bodega,Pub,Bar,Bakery,Discount Store,Rental Car Location,Italian Restaurant,Arcade,Track
12,City Island,Harbor / Marina,Italian Restaurant,Seafood Restaurant,American Restaurant,Deli / Bodega,Bar,History Museum,Diner,Park,Music Venue
28,Throgs Neck,Italian Restaurant,Pizza Place,Pub,Coffee Shop,Bar,Chinese Restaurant,Sports Bar,Deli / Bodega,Asian Restaurant,Juice Bar
29,Country Club,Italian Restaurant,Sandwich Place,Pizza Place,Playground,Deli / Bodega,Dive Bar,Bakery,Track,BBQ Joint,Chinese Restaurant
34,Belmont,Italian Restaurant,Pizza Place,Bakery,Deli / Bodega,Sandwich Place,Pharmacy,Plaza,Donut Shop,Shoe Store,Dessert Shop
37,Pelham Bay,Italian Restaurant,Convenience Store,Bank,Bakery,Gym / Fitness Center,Salon / Barbershop,Fast Food Restaurant,Sandwich Place,Dive Bar,Pharmacy
39,Edgewater Park,Deli / Bodega,Italian Restaurant,Pizza Place,Coffee Shop,Bar,Frozen Yogurt Shop,Mobile Phone Shop,Spa,Spanish Restaurant,Latin American Restaurant
46,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Cosmetics Shop,Bagel Shop,Bar,Greek Restaurant,Chinese Restaurant,Bakery,Donut Shop


## Cluster 2

In [55]:
bxbk_merged.loc[bxbk_merged['Cluster Labels'] == 1, bxbk_merged.columns[[1] + list(range(5, bxbk_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Gym,Donut Shop,Athletics & Sports,Fish Market,Farm,Farmers Market,Fast Food Restaurant


## Cluster 3

In [56]:
bxbk_merged.loc[bxbk_merged['Cluster Labels'] == 2, bxbk_merged.columns[[1] + list(range(5, bxbk_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Mattress Store,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Supermarket,Men's Store,Spanish Restaurant,Pharmacy,Mobile Phone Shop,Moving Target
8,Norwood,Pizza Place,Park,Pharmacy,Mobile Phone Shop,Supermarket,Chinese Restaurant,Spanish Restaurant,Fast Food Restaurant,Deli / Bodega,Donut Shop
10,Baychester,Pharmacy,Furniture / Home Store,Bus Station,Bank,Coffee Shop,Discount Store,Supermarket,Bakery,Department Store,Spanish Restaurant
11,Pelham Parkway,Sandwich Place,Pizza Place,Italian Restaurant,Liquor Store,Donut Shop,Chinese Restaurant,Ice Cream Shop,Bank,Bakery,Sushi Restaurant
13,Bedford Park,Deli / Bodega,Supermarket,Pizza Place,Sandwich Place,Mexican Restaurant,Diner,Chinese Restaurant,Food Truck,Donut Shop,Garden
14,University Heights,Pizza Place,Grocery Store,Fried Chicken Joint,Chinese Restaurant,Fast Food Restaurant,Spanish Restaurant,Bus Station,Playground,BBQ Joint,Bakery
15,Morris Heights,Spanish Restaurant,Food Truck,Pizza Place,Bar,Recreation Center,Bus Station,Music Venue,Metro Station,Supermarket,Grocery Store
16,Fordham,Fried Chicken Joint,Pizza Place,Spanish Restaurant,Mobile Phone Shop,Shoe Store,Pharmacy,Diner,Supplement Shop,Donut Shop,Grocery Store
17,East Tremont,Pizza Place,Sandwich Place,Bank,Breakfast Spot,Supermarket,Fried Chicken Joint,Mobile Phone Shop,Shoe Store,Fast Food Restaurant,Café
18,West Farms,Zoo,Bus Station,Pizza Place,Park,Sandwich Place,Fast Food Restaurant,Breakfast Spot,Mobile Phone Shop,Liquor Store,Supplement Shop


## Cluster 4

In [57]:
bxbk_merged.loc[bxbk_merged['Cluster Labels'] == 3, bxbk_merged.columns[[1] + list(range(5, bxbk_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Playground,Park,Café,River,Home Service,Art Gallery,Bus Station,Coffee Shop,Pizza Place
27,Clason Point,Park,Pool,Scenic Lookout,Boat or Ferry,River,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
35,Spuyten Duyvil,Park,Dog Run,Bank,Intersection,Train Station,Pizza Place,Diner,Pharmacy,Chinese Restaurant,Café
76,Mill Island,Pool,Burger Joint,Seafood Restaurant,Harbor / Marina,Park,Fast Food Restaurant,Gym Pool,Playground,Flower Shop,Farm
85,Sea Gate,Beach,Bus Station,Spa,Supermarket,Park,Fishing Store,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


## Cluster 5

In [58]:
bxbk_merged.loc[bxbk_merged['Cluster Labels'] == 4, bxbk_merged.columns[[1] + list(range(5, bxbk_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Supermarket,Caribbean Restaurant,Pharmacy,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Health & Beauty Service,Gas Station,Fried Chicken Joint,Donut Shop
2,Eastchester,Bus Station,Caribbean Restaurant,Fast Food Restaurant,Women's Store,Grocery Store,Pizza Place,Diner,Bus Stop,Shopping Mall,Business Service
9,Williamsbridge,Pizza Place,Caribbean Restaurant,Pharmacy,Discount Store,Bar,Soup Place,Fried Chicken Joint,Supermarket,Nightclub,Fast Food Restaurant
41,Olinville,Donut Shop,Pharmacy,Fast Food Restaurant,Sandwich Place,Caribbean Restaurant,Bank,Supermarket,Pizza Place,Food,Mobile Phone Shop
45,Edenwald,Supermarket,Caribbean Restaurant,Pharmacy,Sandwich Place,Donut Shop,Nightclub,Bus Station,Fish Market,Fast Food Restaurant,Grocery Store
56,East Flatbush,Caribbean Restaurant,Liquor Store,Lounge,Performing Arts Venue,Deli / Bodega,Fruit & Vegetable Store,Miscellaneous Shop,Supermarket,Grocery Store,Chinese Restaurant
74,Canarsie,Caribbean Restaurant,Bank,Gym,Fast Food Restaurant,Athletics & Sports,Diner,Pizza Place,Bus Line,Food,Grocery Store
75,Flatlands,Pharmacy,Fast Food Restaurant,Liquor Store,Chinese Restaurant,Lounge,Caribbean Restaurant,Sandwich Place,Deli / Bodega,Nightclub,Electronics Store
223,Rugby,Caribbean Restaurant,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Seafood Restaurant,Chinese Restaurant,Pizza Place,Donut Shop,Pharmacy,Bank
259,Remsen Village,Caribbean Restaurant,Fast Food Restaurant,Food,Donut Shop,Deli / Bodega,Liquor Store,Fried Chicken Joint,Discount Store,Sandwich Place,Salad Place
